# Some configurations

In [1]:
import warnings
warnings.filterwarnings('ignore')

# Load test data

## media's data

In [2]:
# import os

# test_data_dir = "data/test"

# filenames = [filename for filename in os.listdir(test_data_dir) if filename.endswith('.amr')]
# print(filenames[:5])

## Common voice 17.0

In [3]:
import os
import pandas as pd

test_data_dir = "data/cv-corpus-17.0-2024-03-15/zh-HK/clips"
test_data_path = "data/cv-corpus-17.0-2024-03-15/zh-HK/test.tsv"

test_data = pd.read_csv(test_data_path, sep='\t')
print(test_data.shape)
test_data.head()

(5595, 13)


,client_id,path,sentence_id,sentence,sentence_domain,up_votes,down_votes,age,gender,accents,variant,locale,segment
0,01bde5386cf720d07d14c8afe4f3a15f898e124637bc88...,common_voice_zh-HK_22943345.mp3,31b20bb13ed38bced6db246b766710d8af666812f6ae7a...,杞人嘅朋友嘆咗一口氣,NaN,2,0,NaN,NaN,NaN,NaN,zh-HK,NaN
1,072ccb7ef5a59a9c5cf2cfd6e1ce4d181bb1463d829edb...,common_voice_zh-HK_22261563.mp3,aafc073108d4069455d34cc2f1a264bcd1ab3b1b8c5082...,聚星樓,NaN,2,1,NaN,NaN,NaN,NaN,zh-HK,NaN
2,0d40495af6f380e52ff8b0eabeb10948c2fcf0bddbe7ac...,common_voice_zh-HK_38078922.mp3,09cda508dcb6f687732e2e1ad57aa8c176581c4ad30d75...,八,NaN,2,1,NaN,NaN,NaN,NaN,zh-HK,Benchmark
3,188aab4ec1f20fe4d0a33ed59e6b09ec9266755aa22e82...,common_voice_zh-HK_22430107.mp3,6e33bff32febfd4dff9cd1b6adb3bedf21125d6b07bc3e...,泥水佬開門口過得人過得自己,NaN,2,0,NaN,NaN,NaN,NaN,zh-HK,NaN
4,193ff377c8fdbb940510204b60af2a295fd6da9e6bdf49...,common_voice_zh-HK_22268464.mp3,0050418d1e8fcdde1109a171e662a245e6427c805f7e86...,嗰次喺跑馬地協和里見到有雞蛋仔格仔餅賣,NaN,2,0,thirties,male_masculine,NaN,NaN,zh-HK,NaN


In [4]:
num_test = 10000
filenames = test_data['path'][:num_test]
filenames[:5]

0    common_voice_zh-HK_22943345.mp3
1    common_voice_zh-HK_22261563.mp3
2    common_voice_zh-HK_38078922.mp3
3    common_voice_zh-HK_22430107.mp3
4    common_voice_zh-HK_22268464.mp3
Name: path, dtype: object

# Test whisper models

In [5]:
## old version
# import whisper

# model = whisper.load_model("base")

# audio_path = "data/test/f57c791f-074f-4706-94ca-a9b4bcab7ce4.amr"

# # load audio and pad/trim it to fit 30 seconds
# audio = whisper.load_audio(audio_path)
# audio = whisper.pad_or_trim(audio)

# # make log-Mel spectrogram and move to the same device as the model
# mel = whisper.log_mel_spectrogram(audio).to(model.device)

# # detect the spoken language
# _, probs = model.detect_language(mel)
# print(f"Detected language: {max(probs, key=probs.get)}")

# # decode the audio
# options = whisper.DecodingOptions()
# result = whisper.decode(model, mel, options)

# # print the recognized text
# print(result.text)

# def whisper_decode(path):
#     # load audio and pad/trim it to fit 30 seconds
#     audio = whisper.load_audio(path)
#     audio = whisper.pad_or_trim(audio)

#     # make log-Mel spectrogram and move to the same device as the model
#     mel = whisper.log_mel_spectrogram(audio).to(model.device)

#     # detect the spoken language
#     _, probs = model.detect_language(mel)
#     # print(f"Detected language: {max(probs, key=probs.get)}")

#     # decode the audio
#     options = whisper.DecodingOptions()
#     result = whisper.decode(model, mel, options)

#     # print the recognized text
#     # print(result.text)
    
#     return {'text': result.text, 'language': max(probs, key=probs.get)}

In [6]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_dataset

device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32
print(f"device: {device}")

def initialize_pipe(model_id):
    
    model = AutoModelForSpeechSeq2Seq.from_pretrained(model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True, attn_implementation="flash_attention_2") # use_flash_attention_2=True) # 

    model.to(device)

    if hasattr(model.generation_config, "no_timestamps_token_id"):
        return_timestamps = True
    else:
        return_timestamps = False
        
    processor = AutoProcessor.from_pretrained(model_id)

    pipe = pipeline(
        "automatic-speech-recognition",
        model=model,
        tokenizer=processor.tokenizer,
        feature_extractor=processor.feature_extractor,
        max_new_tokens=128,
        chunk_length_s=30,
        batch_size=16,
        return_timestamps=return_timestamps,
        torch_dtype=torch_dtype,
        device=device,
    )
    
    return pipe

device: cuda:0


## whisper medium 

In [7]:
# model_id = "/home/ec2-user/SageMaker/efs/Models/whisper-medium"
model_id = "/home/ec2-user/SageMaker/efs/Models/whisper-small-cantonese"

pipe = initialize_pipe(model_id)

sample = os.path.join(test_data_dir, filenames[0])

result = pipe(sample)
print(result["text"])

You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


杞人憂天嘅朋友 嘆咗一口氣


In [8]:
import pandas as pd
from tqdm import tqdm
import time

results = []

for filename in tqdm(filenames, total=len(filenames)):
    start_time = time.time()
    result = pipe(os.path.join(test_data_dir, filename)) # , generate_kwargs={"language": "chinese"})
    time_cost = time.time() - start_time
    result['filename'] = filename
    result['time_cost'] = round(time_cost, 2)
    results.append(result)

100%|██████████| 500/500 [02:25<00:00,  3.45it/s]


In [9]:
df_pred_medium = pd.DataFrame(results)
df_pred_medium.rename(columns={'text': 'pred_medium', 'chunks': 'chunks_medium', 'time_cost': 'time_cost_medium'}, inplace=True)
df_pred_medium.head()

,pred_medium,filename,time_cost_medium
0,杞人憂天嘅朋友 嘆咗一口氣,common_voice_zh-HK_22943345.mp3,0.37
1,聚星樓,common_voice_zh-HK_22261563.mp3,0.24
2,八,common_voice_zh-HK_38078922.mp3,0.19
3,泥水路開門口過得人過得自己,common_voice_zh-HK_22430107.mp3,0.31
4,嗰次喺跑馬地協和你見到有雞蛋仔格仔餅賣,common_voice_zh-HK_22268464.mp3,0.45


## whisper large

In [10]:
model_id = "/home/ec2-user/SageMaker/efs/Models/whisper-large-v3"

pipe = initialize_pipe(model_id)

sample = os.path.join(test_data_dir, filenames[0])

result = pipe(sample)
print(result["text"])

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.


幾人幽聽的朋友嘆了一口氣


In [11]:
import pandas as pd
from tqdm import tqdm
import time

results_large = []

for filename in tqdm(filenames, total=len(filenames)):
    start_time = time.time()
    result = pipe(os.path.join(test_data_dir, filename), generate_kwargs={"language": "cantonese"})
    time_cost = time.time() - start_time
    result['filename'] = filename
    result['time_cost'] = round(time_cost, 2)
    results_large.append(result)
    
df_pred_large = pd.DataFrame(results_large)
df_pred_large.rename(columns={'text': 'pred_large', 'chunks': 'chunks_large', 'time_cost': 'time_cost_large'}, inplace=True)
df_pred_large.head()

100%|██████████| 500/500 [04:34<00:00,  1.82it/s]


,pred_large,chunks_large,filename,time_cost_large
0,幾人幽聽嘅朋友嘆咗一口氣,"[{'timestamp': (0.0, 4.0), 'text': ' 幾人幽聽嘅朋友嘆咗...",common_voice_zh-HK_22943345.mp3,0.60
1,最升樓,"[{'timestamp': (0.0, 2.12), 'text': ' 最升樓'}]",common_voice_zh-HK_22261563.mp3,0.34
2,爸,"[{'timestamp': (0.0, 1.84), 'text': ' 爸'}]",common_voice_zh-HK_38078922.mp3,0.27
3,泥水路開門口過得人過得自己,"[{'timestamp': (0.0, 8.52), 'text': ' 泥水路開門口過得...",common_voice_zh-HK_22430107.mp3,0.54
4,嗰次喺跑馬地協和里見到有雞蛋仔格仔餅埋,"[{'timestamp': (0.0, 8.38), 'text': ' 嗰次喺跑馬地協和...",common_voice_zh-HK_22268464.mp3,0.81


In [12]:
## convert whisper-large-v3 to faster-whisper-large-v3

# !ct2-transformers-converter --model openai/whisper-large-v3 --output_dir faster-whisper-large-v3 \
#     --copy_files tokenizer.json preprocessor_config.json --quantization float16

## faster_whisper_v3

In [13]:
from faster_whisper import WhisperModel

model_id_faster =  "/home/ec2-user/SageMaker/efs/Models/faster-whisper-large-v3"
model_faster = WhisperModel(model_id_faster)

segments, info = model_faster.transcribe(sample, language="yue") # 
print("Detected language '%s' with probability %f" % (info.language, info.language_probability))
for segment in segments:
    print("[%.2fs -> %.2fs] %s" % (segment.start, segment.end, segment.text))
    print()

Detected language 'yue' with probability 1.000000
[0.00s -> 3.94s]  幾人幽聽嘅朋友嘆咗一口氣



In [14]:
import time

results = []

for filename in tqdm(filenames, total=len(filenames)):
    result = {}
    start_time = time.time()
    segments, info = model_faster.transcribe(os.path.join(test_data_dir, filename), language="yue")
    result['pred_faster'] = ''.join([segment.text for segment in segments])
    time_cost = time.time() - start_time
    result['filename'] = filename
    result['time_cost_faster'] = round(time_cost, 2)
    results.append(result)

df_pred_faster = pd.DataFrame(results)
df_pred_faster.head()

100%|██████████| 500/500 [04:42<00:00,  1.77it/s]


,pred_faster,filename,time_cost_faster
0,幾人幽聽嘅朋友嘆咗一口氣,common_voice_zh-HK_22943345.mp3,0.53
1,墜星樓,common_voice_zh-HK_22261563.mp3,0.53
2,爸,common_voice_zh-HK_38078922.mp3,1.09
3,泥水路開門口過得人過得自己,common_voice_zh-HK_22430107.mp3,0.51
4,嗰次喺跑馬地協和里見到有雞蛋仔格仔餅賣,common_voice_zh-HK_22268464.mp3,0.64


## combine all true and pred results

In [15]:
# df_true = pd.read_csv("./data/语料.csv")
# df_all = pd.merge(df_true, df_pred_medium, left_on='record_file_id', right_on='filename')

df_true = pd.read_csv(test_data_path, sep='\t')
df_true = df_true[:num_test]
df_all = pd.merge(df_true, df_pred_medium, left_on='path', right_on='filename')

In [16]:
df_all.head(10)

,client_id,path,sentence_id,sentence,sentence_domain,up_votes,down_votes,age,gender,accents,variant,locale,segment,pred_medium,filename,time_cost_medium
0,01bde5386cf720d07d14c8afe4f3a15f898e124637bc88...,common_voice_zh-HK_22943345.mp3,31b20bb13ed38bced6db246b766710d8af666812f6ae7a...,杞人嘅朋友嘆咗一口氣,NaN,2,0,NaN,NaN,NaN,NaN,zh-HK,NaN,杞人憂天嘅朋友 嘆咗一口氣,common_voice_zh-HK_22943345.mp3,0.37
1,072ccb7ef5a59a9c5cf2cfd6e1ce4d181bb1463d829edb...,common_voice_zh-HK_22261563.mp3,aafc073108d4069455d34cc2f1a264bcd1ab3b1b8c5082...,聚星樓,NaN,2,1,NaN,NaN,NaN,NaN,zh-HK,NaN,聚星樓,common_voice_zh-HK_22261563.mp3,0.24
2,0d40495af6f380e52ff8b0eabeb10948c2fcf0bddbe7ac...,common_voice_zh-HK_38078922.mp3,09cda508dcb6f687732e2e1ad57aa8c176581c4ad30d75...,八,NaN,2,1,NaN,NaN,NaN,NaN,zh-HK,Benchmark,八,common_voice_zh-HK_38078922.mp3,0.19
3,188aab4ec1f20fe4d0a33ed59e6b09ec9266755aa22e82...,common_voice_zh-HK_22430107.mp3,6e33bff32febfd4dff9cd1b6adb3bedf21125d6b07bc3e...,泥水佬開門口過得人過得自己,NaN,2,0,NaN,NaN,NaN,NaN,zh-HK,NaN,泥水路開門口過得人過得自己,common_voice_zh-HK_22430107.mp3,0.31
4,193ff377c8fdbb940510204b60af2a295fd6da9e6bdf49...,common_voice_zh-HK_22268464.mp3,0050418d1e8fcdde1109a171e662a245e6427c805f7e86...,嗰次喺跑馬地協和里見到有雞蛋仔格仔餅賣,NaN,2,0,thirties,male_masculine,NaN,NaN,zh-HK,NaN,嗰次喺跑馬地協和你見到有雞蛋仔格仔餅賣,common_voice_zh-HK_22268464.mp3,0.45
5,1c98e792d45e00fc8766f89553a8c0dfee3e3c41d8466e...,common_voice_zh-HK_22279419.mp3,5e478dd69d058f0d2325f13c9ef7517d23ff6598097d4c...,學生話佢知白沙山路係喺大棠，但係唔知有冇小巴經嗰度,NaN,2,1,NaN,NaN,NaN,NaN,zh-HK,NaN,學生話佢知白沙山路係喺大堂，但係唔知有冇小巴經嗰度,common_voice_zh-HK_22279419.mp3,0.57
6,1e57dd93068fb68d892a0a50d8ee7268ced460d4931d75...,common_voice_zh-HK_22262988.mp3,c15dd07df8f462428711a876aef2f9f89e7d324b417fd1...,有個老人去左坪洲永安橫街食齋,NaN,2,1,NaN,NaN,NaN,NaN,zh-HK,NaN,有個老人去左坪洲永安橫街食齋,common_voice_zh-HK_22262988.mp3,0.38
7,1e701a6916f5fc82d63ae7147858dd27bb82b4649f9620...,common_voice_zh-HK_22945665.mp3,1afa17ed1b7682737f3ecb27825f27f2ecff5513a5e287...,說我自找,NaN,2,0,NaN,NaN,NaN,NaN,zh-HK,NaN,算我自找,common_voice_zh-HK_22945665.mp3,0.20
8,1ebc1e4fe3753c0b5ad64ab52b36bee2d9211a84af7d5c...,common_voice_zh-HK_22261557.mp3,9c15748502879516ac84aa93342ff10c9a7aed83cb0889...,選定了心儀嘅機構先查詢個人實際年利率，咁會比較明智,NaN,2,0,NaN,NaN,NaN,NaN,zh-HK,NaN,選定了心儀嘅機構先查詢個人實際連利率噉會比較明智,common_voice_zh-HK_22261557.mp3,0.90
9,2431d2121d283329820cd2eba62f0ac643821bb0de4481...,common_voice_zh-HK_26520520.mp3,63d11a481ff8d9c3ae2080ae54b93e76c6144fb553871c...,唔係,NaN,2,0,NaN,NaN,NaN,NaN,zh-HK,Benchmark,唔係,common_voice_zh-HK_26520520.mp3,1.40


In [17]:
df_all = pd.merge(df_all, df_pred_large, left_on='filename', right_on='filename')
df_all.head()

,client_id,path,sentence_id,sentence,sentence_domain,up_votes,down_votes,age,gender,accents,variant,locale,segment,pred_medium,filename,time_cost_medium,pred_large,chunks_large,time_cost_large
0,01bde5386cf720d07d14c8afe4f3a15f898e124637bc88...,common_voice_zh-HK_22943345.mp3,31b20bb13ed38bced6db246b766710d8af666812f6ae7a...,杞人嘅朋友嘆咗一口氣,NaN,2,0,NaN,NaN,NaN,NaN,zh-HK,NaN,杞人憂天嘅朋友 嘆咗一口氣,common_voice_zh-HK_22943345.mp3,0.37,幾人幽聽嘅朋友嘆咗一口氣,"[{'timestamp': (0.0, 4.0), 'text': ' 幾人幽聽嘅朋友嘆咗...",0.60
1,072ccb7ef5a59a9c5cf2cfd6e1ce4d181bb1463d829edb...,common_voice_zh-HK_22261563.mp3,aafc073108d4069455d34cc2f1a264bcd1ab3b1b8c5082...,聚星樓,NaN,2,1,NaN,NaN,NaN,NaN,zh-HK,NaN,聚星樓,common_voice_zh-HK_22261563.mp3,0.24,最升樓,"[{'timestamp': (0.0, 2.12), 'text': ' 最升樓'}]",0.34
2,0d40495af6f380e52ff8b0eabeb10948c2fcf0bddbe7ac...,common_voice_zh-HK_38078922.mp3,09cda508dcb6f687732e2e1ad57aa8c176581c4ad30d75...,八,NaN,2,1,NaN,NaN,NaN,NaN,zh-HK,Benchmark,八,common_voice_zh-HK_38078922.mp3,0.19,爸,"[{'timestamp': (0.0, 1.84), 'text': ' 爸'}]",0.27
3,188aab4ec1f20fe4d0a33ed59e6b09ec9266755aa22e82...,common_voice_zh-HK_22430107.mp3,6e33bff32febfd4dff9cd1b6adb3bedf21125d6b07bc3e...,泥水佬開門口過得人過得自己,NaN,2,0,NaN,NaN,NaN,NaN,zh-HK,NaN,泥水路開門口過得人過得自己,common_voice_zh-HK_22430107.mp3,0.31,泥水路開門口過得人過得自己,"[{'timestamp': (0.0, 8.52), 'text': ' 泥水路開門口過得...",0.54
4,193ff377c8fdbb940510204b60af2a295fd6da9e6bdf49...,common_voice_zh-HK_22268464.mp3,0050418d1e8fcdde1109a171e662a245e6427c805f7e86...,嗰次喺跑馬地協和里見到有雞蛋仔格仔餅賣,NaN,2,0,thirties,male_masculine,NaN,NaN,zh-HK,NaN,嗰次喺跑馬地協和你見到有雞蛋仔格仔餅賣,common_voice_zh-HK_22268464.mp3,0.45,嗰次喺跑馬地協和里見到有雞蛋仔格仔餅埋,"[{'timestamp': (0.0, 8.38), 'text': ' 嗰次喺跑馬地協和...",0.81


In [18]:
df_all = pd.merge(df_all, df_pred_faster, left_on='filename', right_on='filename')
df_all.head()

,client_id,path,sentence_id,sentence,sentence_domain,up_votes,down_votes,age,gender,accents,...,locale,segment,pred_medium,filename,time_cost_medium,pred_large,chunks_large,time_cost_large,pred_faster,time_cost_faster
0,01bde5386cf720d07d14c8afe4f3a15f898e124637bc88...,common_voice_zh-HK_22943345.mp3,31b20bb13ed38bced6db246b766710d8af666812f6ae7a...,杞人嘅朋友嘆咗一口氣,NaN,2,0,NaN,NaN,NaN,...,zh-HK,NaN,杞人憂天嘅朋友 嘆咗一口氣,common_voice_zh-HK_22943345.mp3,0.37,幾人幽聽嘅朋友嘆咗一口氣,"[{'timestamp': (0.0, 4.0), 'text': ' 幾人幽聽嘅朋友嘆咗...",0.60,幾人幽聽嘅朋友嘆咗一口氣,0.53
1,072ccb7ef5a59a9c5cf2cfd6e1ce4d181bb1463d829edb...,common_voice_zh-HK_22261563.mp3,aafc073108d4069455d34cc2f1a264bcd1ab3b1b8c5082...,聚星樓,NaN,2,1,NaN,NaN,NaN,...,zh-HK,NaN,聚星樓,common_voice_zh-HK_22261563.mp3,0.24,最升樓,"[{'timestamp': (0.0, 2.12), 'text': ' 最升樓'}]",0.34,墜星樓,0.53
2,0d40495af6f380e52ff8b0eabeb10948c2fcf0bddbe7ac...,common_voice_zh-HK_38078922.mp3,09cda508dcb6f687732e2e1ad57aa8c176581c4ad30d75...,八,NaN,2,1,NaN,NaN,NaN,...,zh-HK,Benchmark,八,common_voice_zh-HK_38078922.mp3,0.19,爸,"[{'timestamp': (0.0, 1.84), 'text': ' 爸'}]",0.27,爸,1.09
3,188aab4ec1f20fe4d0a33ed59e6b09ec9266755aa22e82...,common_voice_zh-HK_22430107.mp3,6e33bff32febfd4dff9cd1b6adb3bedf21125d6b07bc3e...,泥水佬開門口過得人過得自己,NaN,2,0,NaN,NaN,NaN,...,zh-HK,NaN,泥水路開門口過得人過得自己,common_voice_zh-HK_22430107.mp3,0.31,泥水路開門口過得人過得自己,"[{'timestamp': (0.0, 8.52), 'text': ' 泥水路開門口過得...",0.54,泥水路開門口過得人過得自己,0.51
4,193ff377c8fdbb940510204b60af2a295fd6da9e6bdf49...,common_voice_zh-HK_22268464.mp3,0050418d1e8fcdde1109a171e662a245e6427c805f7e86...,嗰次喺跑馬地協和里見到有雞蛋仔格仔餅賣,NaN,2,0,thirties,male_masculine,NaN,...,zh-HK,NaN,嗰次喺跑馬地協和你見到有雞蛋仔格仔餅賣,common_voice_zh-HK_22268464.mp3,0.45,嗰次喺跑馬地協和里見到有雞蛋仔格仔餅埋,"[{'timestamp': (0.0, 8.38), 'text': ' 嗰次喺跑馬地協和...",0.81,嗰次喺跑馬地協和里見到有雞蛋仔格仔餅賣,0.64


# Calculate Character Error Rate (CER)

In [19]:
import re

class ChineseTextNormalizer:
    def __init__(self, split_letters: bool = False):

        self.split_letters = split_letters

    def __call__(self, s: str):
        s = s.lower()
        # s = re.sub(r"[<\[][^>\]]*[>\]]", "", s)  # remove words between brackets
        # s = re.sub(r"\(([^)]+?)\)", "", s)  # remove words between parenthesis
        # s = self.clean(s).lower()

        # if self.split_letters:
        #     s = " ".join(regex.findall(r"\X", s, regex.U))

        s = re.sub(
            r"\s+", "", s
        )  # remove any successive whitespace characters
        
        return s

In [20]:
import jiwer
from whisper.normalizers import BasicTextNormalizer, EnglishTextNormalizer

# normalizer = EnglishTextNormalizer()
# normalizer = BasicTextNormalizer()
normalizer = ChineseTextNormalizer()

In [21]:
for i, row in tqdm(df_all.iterrows(), total=df_all.shape[0]):
    
    # content_clean = normalizer(row['content'])
    content_clean = normalizer(row['sentence'])
    pred_medium_clean = normalizer(row['pred_medium'])
    pred_large_clean = normalizer(row['pred_large'])
    pred_faster_clean = normalizer(row['pred_faster'])
    
    df_all.loc[i, 'content_clean'] = content_clean
    df_all.loc[i, 'pred_medium_clean'] = pred_medium_clean
    df_all.loc[i, 'pred_large_clean'] = pred_large_clean
    df_all.loc[i, 'pred_faster_clean'] = pred_faster_clean

# new_columns = ['category_name', 'user_id', 'content', 'content_clean', 'pred_medium', 'pred_medium_clean', 'pred_large', 'pred_large_clean', 'pred_faster', 'pred_faster_clean', 'time_cost_medium', 'time_cost_large', 'time_cost_faster', 'chunks_medium', 'chunks_large', 'record_file_id',  'filename']
new_columns = ['sentence', 'content_clean', 'pred_medium', 'pred_medium_clean', 'pred_large', 'pred_large_clean', 'pred_faster', 'pred_faster_clean', 'time_cost_medium', 'time_cost_large', 'time_cost_faster', 'filename']

df_all = df_all[new_columns]
df_all.head(10)

100%|██████████| 500/500 [00:00<00:00, 1746.02it/s]


,sentence,content_clean,pred_medium,pred_medium_clean,pred_large,pred_large_clean,pred_faster,pred_faster_clean,time_cost_medium,time_cost_large,time_cost_faster,filename
0,杞人嘅朋友嘆咗一口氣,杞人嘅朋友嘆咗一口氣,杞人憂天嘅朋友 嘆咗一口氣,杞人憂天嘅朋友嘆咗一口氣,幾人幽聽嘅朋友嘆咗一口氣,幾人幽聽嘅朋友嘆咗一口氣,幾人幽聽嘅朋友嘆咗一口氣,幾人幽聽嘅朋友嘆咗一口氣,0.37,0.60,0.53,common_voice_zh-HK_22943345.mp3
1,聚星樓,聚星樓,聚星樓,聚星樓,最升樓,最升樓,墜星樓,墜星樓,0.24,0.34,0.53,common_voice_zh-HK_22261563.mp3
2,八,八,八,八,爸,爸,爸,爸,0.19,0.27,1.09,common_voice_zh-HK_38078922.mp3
3,泥水佬開門口過得人過得自己,泥水佬開門口過得人過得自己,泥水路開門口過得人過得自己,泥水路開門口過得人過得自己,泥水路開門口過得人過得自己,泥水路開門口過得人過得自己,泥水路開門口過得人過得自己,泥水路開門口過得人過得自己,0.31,0.54,0.51,common_voice_zh-HK_22430107.mp3
4,嗰次喺跑馬地協和里見到有雞蛋仔格仔餅賣,嗰次喺跑馬地協和里見到有雞蛋仔格仔餅賣,嗰次喺跑馬地協和你見到有雞蛋仔格仔餅賣,嗰次喺跑馬地協和你見到有雞蛋仔格仔餅賣,嗰次喺跑馬地協和里見到有雞蛋仔格仔餅埋,嗰次喺跑馬地協和里見到有雞蛋仔格仔餅埋,嗰次喺跑馬地協和里見到有雞蛋仔格仔餅賣,嗰次喺跑馬地協和里見到有雞蛋仔格仔餅賣,0.45,0.81,0.64,common_voice_zh-HK_22268464.mp3
5,學生話佢知白沙山路係喺大棠，但係唔知有冇小巴經嗰度,學生話佢知白沙山路係喺大棠，但係唔知有冇小巴經嗰度,學生話佢知白沙山路係喺大堂，但係唔知有冇小巴經嗰度,學生話佢知白沙山路係喺大堂，但係唔知有冇小巴經嗰度,"學生話佢知白沙山路係喺大堂,但係唔知有無小巴經嗰度。","學生話佢知白沙山路係喺大堂,但係唔知有無小巴經嗰度。","學生話佢知白沙山路係喺大堂,但係唔知有無小巴經嗰度。","學生話佢知白沙山路係喺大堂,但係唔知有無小巴經嗰度。",0.57,0.98,0.70,common_voice_zh-HK_22279419.mp3
6,有個老人去左坪洲永安橫街食齋,有個老人去左坪洲永安橫街食齋,有個老人去左坪洲永安橫街食齋,有個老人去左坪洲永安橫街食齋,有個老人去咗平洲永安橫街食齋,有個老人去咗平洲永安橫街食齋,有個老人去咗平洲永安橫街食齋,有個老人去咗平洲永安橫街食齋,0.38,0.61,0.53,common_voice_zh-HK_22262988.mp3
7,說我自找,說我自找,算我自找,算我自找,酸果自找,酸果自找,酸哥自找,酸哥自找,0.20,0.36,0.41,common_voice_zh-HK_22945665.mp3
8,選定了心儀嘅機構先查詢個人實際年利率，咁會比較明智,選定了心儀嘅機構先查詢個人實際年利率，咁會比較明智,選定了心儀嘅機構先查詢個人實際連利率噉會比較明智,選定了心儀嘅機構先查詢個人實際連利率噉會比較明智,"選定了心儀的機構先查詢個人實際年利率,這樣會比較明智","選定了心儀的機構先查詢個人實際年利率,這樣會比較明智","選定了心儀的機構先查詢個人實際年利率,這樣會比較明智。","選定了心儀的機構先查詢個人實際年利率,這樣會比較明智。",0.90,0.85,0.72,common_voice_zh-HK_22261557.mp3
9,唔係,唔係,唔係,唔係,唔係,唔係,不是,不是,1.40,0.30,0.37,common_voice_zh-HK_26520520.mp3


In [22]:
df_all.fillna('', inplace=True)

In [23]:
cer_medium = jiwer.cer(list(df_all["content_clean"]), list(df_all["pred_medium_clean"]))
cer_large = jiwer.cer(list(df_all["content_clean"]), list(df_all["pred_large_clean"]))
cer_faster = jiwer.cer(list(df_all["content_clean"]), list(df_all["pred_faster_clean"]))

print(f"CER_medium: {cer_medium * 100:.2f} %, CER_large: {cer_large * 100:.2f} %, CER_faster: {cer_faster * 100:.2f} %")

CER_medium: 13.67 %, CER_large: 23.67 %, CER_faster: 22.33 %


In [24]:
df_all[['time_cost_medium', 'time_cost_large', 'time_cost_faster']].describe(percentiles=[0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 0.99])

,time_cost_medium,time_cost_large,time_cost_faster
count,500.000000,500.000000,500.000000
mean,0.289340,0.548300,0.563620
std,0.148478,0.193547,0.427192
min,0.150000,0.240000,0.360000
50%,0.250000,0.510000,0.520000
60%,0.290000,0.570000,0.550000
70%,0.330000,0.660000,0.590000
80%,0.370000,0.752000,0.620000
90%,0.400000,0.820000,0.670000
95%,0.430000,0.880000,0.721000
